# Clearwater Modules Architecture

**Author:** Xavier Nogueira
**Overview:**
* How the code organizes a model (static, dynamic, and state variables).
* How the base.Model class handle all shared functionality.

In [2]:
import clearwater_modules_python

AttributeError: module 'clearwater_modules_python.tsm.processes' has no attribute 'density_air_saturation'